In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
#sys.path.append("..")
import module.master as master
import module.model_select_save as select

In [2]:
place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass
    

# use_model_dfの作成と保存

In [3]:
#データ格納用のディレクトリ作り
version='V3_1'#バージョン
years=[2018,2019,2020]#使用する三年分のデータ（古い順で！！！）

place_master=master.get_place_master()
for place in place_master.items():
    place_name=place[1]
    print('=========================================================================================================================')
    print(place_name)
    ################################################========================================================================================================================
    #各会場の学習データの読み込みと加工
    expand_filepath="../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_3year_expand_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    expand_score_df=pd.read_csv(expand_filepath)#3年分のデータの読み込み
    expand_score_df=expand_score_df.drop(["Unnamed: 0"],axis=1)#余計な列の削除
    input_expand_score=len(expand_score_df)#データの件数
    expand_score_df=expand_score_df.dropna()#収益の計算ができない部分がnanになって排出されてる臭いので削除
    after_expand_score=len(expand_score_df)#データの件数
    
    score_filepath="../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    score_df=pd.read_csv(score_filepath)#２年分の本番予測用のデータ
    input_score=len(score_df)#データの件数
    score_df=score_df.drop(["Unnamed: 0"],axis=1)#余計な列の削除
    score_df=score_df.drop(["gain_year3"],axis=1)#余計な列の削除
    score_df=score_df.dropna()#収益の計算ができない部分がnanになって排出されてる臭いので削除
    after_score=len(score_df)#データの件数
    print("input_expand:{}_____droped_expand:{}".format(input_expand_score,after_expand_score))
    print("input:{}_____droped:{}".format(input_score,after_score))
    #select.model_selection_save(expand_score_df,score_df,place_name,version,th=0.75)
    select.model_selection_save(expand_score_df,score_df,place_name,version,th=0.60)
    #select.model_selection_save(expand_score_df,score_df,place_name,version,th=0.50)

kiryu
input_expand:31752_____droped_expand:31752
input:31752_____droped:31752
train:0.9009881500728317__test:0.8872618485277909
CHECK::::train_precision:84.51160280626013__test_precision:78.88631090487239
toda
input_expand:31752_____droped_expand:31752
input:31752_____droped:31752
train:0.873627022558167__test:0.8401826484018264
CHECK::::train_precision:90.87677725118483__test_precision:77.97619047619048
edogawa
input_expand:31752_____droped_expand:31752
input:31752_____droped:31752
train:0.861658989803551__test:0.8502598016060463
CHECK::::train_precision:89.18128654970761__test_precision:83.06092124814265
heiwazima
input_expand:31752_____droped_expand:31752
input:31752_____droped:31752
train:0.8900830675957639__test:0.8554558337269721
CHECK::::train_precision:92.62672811059907__test_precision:77.27272727272727
tamagawa
input_expand:31752_____droped_expand:31752
input:31752_____droped:31752
train:0.8327231211369631__test:0.7970398362462604
CHECK::::train_precision:91.26460132046724__te

# 予測時の学習データ作成のためのクラスタリングモデルの保存

In [4]:
#データ格納用のディレクトリ作り
version='V3_1'#バージョン
years=[2018,2019,2020]#使用する三年分のデータ（古い順で！！！）

place_master=master.get_place_master()
for place in place_master.items():
    place_name=place[1]
    print(place_name)
    ################################################========================================================================================================================
    #各会場の学習データの読み込みと加工
    result_filepath="../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    result_base_df=pd.read_csv(result_filepath)
    result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
    select.save_clustar_model(result_base_df,place_name,version)

kiryu
toda
edogawa
heiwazima
tamagawa
hamanako
gamagori
tokoname
tu
mikuni
biwako
suminoe
amagasaki
naruto
marugame
kozima
miyazima
tokuyama
simonoseki
wakamatu
asiya
fukuoka
karatu
omura


# 購買予測に使うもでる（XGboost）の保存

In [5]:
#データ格納用のディレクトリ作り
version='V3_1'#バージョン
years=[2018,2019,2020]#使用する三年分のデータ（古い順で！！！）

place_master=master.get_place_master()
for place in place_master.items():
    place_name=place[1]
    ################################################========================================================================================================================
    #各会場の学習データの読み込みと加工
    result_filepath="../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    result_base_df=pd.read_csv(result_filepath)
    result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
    
    use_model_filepath=dir_path =  "../bot_database/{place_name}/model_score_{place_name}/use_model/use_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#選定されたモデルのリストを出力
    use_model_df=pd.read_csv(use_model_filepath)
    use_model_df=use_model_df.drop(["Unnamed: 0"],axis=1)
    
    
    select.save_model_XGboost_V3_1(result_base_df,use_model_df,years,place_name,version)

kiryu
toda
edogawa
heiwazima
tamagawa
hamanako
gamagori
tokoname
tu
mikuni
biwako
suminoe
amagasaki
naruto
marugame
kozima
miyazima
tokuyama
simonoseki
wakamatu
asiya
fukuoka
karatu
omura


# テスト

In [6]:
def trans_result_com(target_com,trans_base_df):#comをターゲットに合わせて0,1の二値に変換する。
    #学習データのラベル変換==========================================================
    trans_df=trans_base_df.copy()
    #result_train_df=trans_base_df.copy()
    result_arr=[0]*len(trans_df)
    i=0
    for result in trans_df['result_com']:#
        if ((result==target_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    trans_df['result_com']=result_arr
    return trans_df

def pred_th_trans(pred_df,th):#引数として予測結果のdeと、変換したい閾値を渡す。

    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df

def pred_th_trans_com(pred_df,th,target_com):#指定の組のカラムのみを置換。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_{}'.format(target_com)] >= th, 'pred_{}'.format(target_com)] = 1
    trans_df.loc[~(trans_df['pred_{}'.format(target_com)] >=  th), 'pred_{}'.format(target_com)] = 0
    return trans_df

def calc_gain(pred_gain_df):#レース単位であたっているか同課の判別と、当たった場合に得られた配当金を計算する関数
    pred_true_df=pred_gain_df[(pred_gain_df['pred']==1)&(pred_gain_df['trans_result']==1)].copy()
    pred_true_df['hit']=1
    calc_base_df=pred_gain_df.copy()
    calc_base_df['hit']=pred_true_df['hit']
    calc_base_df['gain']=pred_true_df['money']
    calc_base_df=calc_base_df.fillna(0)
    #
    #calc_base_df:予測、変換積みの結果、実際の結果、配当金、収益をすべて表したdf,合計操作は行っていない。
    #
    return calc_base_df

def check_pred_arr(pred1_df,pred2_df):#カラムの中身が同じか比較する関数
    pred_1_vals=[pred1_df[col] for col in pred1_df.columns]
    pred_2_vals=[pred2_df[col] for col in pred2_df.columns]
    for col_name1,col1,col_name2,col2 in zip(pred1_df.columns,pred_1_vals,pred2_df.columns,pred_2_vals):
        if list(col1.values)==list(col1.values):
            print(col_name1,'  and  ',col_name2,'  is same pred \n')
    return None

def pred_th_trans(pred_df,th):#閾値を渡して、その値以上を1、未満を0に置き変える。
    #引数として予測結果のdeと、変換したい閾値を渡す。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df

def add_result_class(model,score_df):#モデルでの予測結果を学習時に使用したパラメータシートに追加して返す関数
    #out_figはオプション、グラフの出力の有り無し
    score_sheet_base=score_df.copy()
    score_sheet_shuffle=score_sheet_base.sample(frac=1, random_state=7)#行をシャッフル
    num_data=len(score_sheet_shuffle)#データの件数

    #前処理    これはクラス分類なので閾値で最新収益を1,0に変換する
    score_sheet_shuffle['gain_label']=0#実際の収益の有無を確認(110を利益有り無しの閾値とする )
    score_sheet_shuffle.loc[score_sheet_shuffle['gain_year3'] < 110, 'gain_label'] =0
    score_sheet_shuffle.loc[score_sheet_shuffle['gain_year3'] >= 110, 'gain_label'] =1
    ex_df=score_sheet_shuffle.copy()#gain_year3の切り抜き用
    score_sheet_shuffle=score_sheet_shuffle.drop(['gain_year3'],axis=1).copy()

    #データの分割========================================================================================================
    test_df = score_sheet_shuffle[int(num_data*0.8):].copy()#20%のデータを検証用データに。
    train_df =  score_sheet_shuffle[:int(num_data*0.8)].copy()#そのほかを学習データに
    gain_s=ex_df[int(num_data*0.8):]['gain_year3'].copy()#検証用データの実収益部分を切り抜く。

    test_x=test_df.drop(['gain_label'],axis=1).copy()
    train_x=train_df.drop(['gain_label'],axis=1).copy()
    test_y=test_df['gain_label']
    train_y=train_df['gain_label']
    test_pred_df=test_df.copy()#予測の結合先を作っておく
    train_pred_df=train_df.copy()#予測の結合先を作っておく

    #予測を出力(確率分布)
    train_pred_proba_arr=model.predict_proba(train_x)
    train_pred_proba=[arr[1] for arr in train_pred_proba_arr]#確率分布の二次元配列なので[1]の部分だけ取り出す
    test_pred_proba_arr = model.predict_proba(test_x)
    test_pred_proba=[arr[1] for arr in test_pred_proba_arr]#確率分布の二次元配列なので[1]の部分だけ取り出す
    test_pred_df["pred_proba"]=test_pred_proba#予測を結合
    train_pred_df["pred_proba"]=train_pred_proba#予測を結合

    #確率分布の閾値で予測をバイナリに変換(仮)
    #test_pred_df=pred_th_trans(test_pred_df,0.5)
    #train_pred_df=pred_th_trans(train_pred_df,0.5)

#     #モデルのスコアの計算
#     col1='pred'
#     col2='gain_label'
#     cross_df,train_precision=making_cross(col1,col2 ,train_pred_df)
#     cross_df,test_precision=making_cross(col1,col2 ,test_pred_df)
#     print("train_precision:{}__test_precision:{}".format(train_precision,test_precision))

    test_pred_df['gain_year3']=gain_s#実際の収益の情報を追加する
    return test_pred_df

def making_cross(col1,col2,base_df,out_cross=0):#クロス収益図を作成してpurecisionを算出する関数
    cross_df=pd.DataFrame(columns=["{}_1".format(col2),"{}_0".format(col2),'sum']
                         ,index=["{}_1".format(col1),"{}_0".format(col1),'sum'])#クロス集計の結果の格納df
    cross_df.at["{}_1".format(col1), "{}_1".format(col2)]=len(base_df[(base_df[col1]==1) & (base_df[col2]==1)])#左上
    cross_df.at["{}_1".format(col1), "{}_0".format(col2)]=len(base_df[(base_df[col1]==1) & (base_df[col2]==0)])#右上
    cross_df.at["{}_0".format(col1), "{}_1".format(col2)]=len(base_df[(base_df[col1]==0) & (base_df[col2]==1)])#左下
    cross_df.at["{}_0".format(col1), "{}_0".format(col2)]=len(base_df[(base_df[col1]==0) & (base_df[col2]==0)])#右下

    cross_df.at["{}_1".format(col1), "sum"]=cross_df.at["{}_1".format(col1), "{}_1".format(col2)]+cross_df.at["{}_1".format(col1), "{}_0".format(col2)]
    cross_df.at["{}_0".format(col1), "sum"]=cross_df.at["{}_0".format(col1), "{}_1".format(col2)]+cross_df.at["{}_0".format(col1), "{}_0".format(col2)]
    cross_df.at["sum", "{}_1".format(col2)]=cross_df.at["{}_1".format(col1), "{}_1".format(col2)]+cross_df.at["{}_0".format(col1), "{}_1".format(col2)]
    cross_df.at["sum", "{}_0".format(col2)]=cross_df.at["{}_1".format(col1), "{}_0".format(col2)]+cross_df.at["{}_0".format(col1), "{}_0".format(col2)]
    if out_cross==1:
        display(cross_df)
    #precisionを算出
    try:
        #precision=(cross_df.at["{}_1".format(col1), "{}_1".format(col2)]/(cross_df.at["{}_1".format(col1), "{}_1".format(col2)]+cross_df.at["{}_1".format(col1), "{}_0".format(col2)]))*100
        precision=(cross_df.at["{}_1".format(col1), "{}_1".format(col2)]/(cross_df.at["{}_1".format(col1),'sum']))*100
    except ZeroDivisionError:
        precision=0

    return cross_df,precision

In [7]:
# def model_selection_save(expand_score_df,score_df,place_name,version,th=0.8):#三年分のスコアシートを与えたら[モデル選定モデル]を作成して保存、かつ実際に使用する購買予測モデルのスコアシートを吐き出してくれる関数(use_model)(モデルの保存も同時に行う。)
#     #expand_score_df:::モデル選定のためのモデル作成の学習データ用の三年間を横に展開したスコアシート
#     #score_df:::実際にも本番で使うモデルを選ぶためのシート（答えなし）
#     score_sheet_base=expand_score_df.copy()
#     score_sheet_shuffle=score_sheet_base.sample(frac=1, random_state=7)#行をシャッフル
#     num_data=len(score_sheet_shuffle)#データの件数

#     #前処理    これはクラス分類なので閾値で最新収益を1,0に変換する
#     score_sheet_shuffle['gain_label']=0#実際の収益の有無を確認(110を利益有り無しの閾値とする )
#     score_sheet_shuffle.loc[score_sheet_shuffle['gain_year3'] < 110, 'gain_label'] =0
#     score_sheet_shuffle.loc[score_sheet_shuffle['gain_year3'] >= 110, 'gain_label'] =1
#     ex_df=score_sheet_shuffle.copy()#gain_year3の切り抜き用
#     score_sheet_shuffle=score_sheet_shuffle.drop(['gain_year3'],axis=1).copy()
#     #選定モデルの学習、保存パート============================================================================================================================================================================
#     #選定モデルの学習、保存パート============================================================================================================================================================================
#     #選定モデルの学習、保存パート============================================================================================================================================================================
#     #選定モデルの学習、保存パート============================================================================================================================================================================
#     #選定モデルの学習、保存パート============================================================================================================================================================================

#     #データの分割========================================================================================================
#     test_df = score_sheet_shuffle[int(num_data*0.8):].copy()#20%のデータを検証用データに。
#     train_df =  score_sheet_shuffle[:int(num_data*0.8)].copy()#そのほかを学習データに
#     gain_s=ex_df[int(num_data*0.8):]['gain_year3'].copy()#検証用データの実収益部分を切り抜く。

#     test_x=test_df.drop(['gain_label'],axis=1).copy()
#     train_x=train_df.drop(['gain_label'],axis=1).copy()
#     test_y=test_df['gain_label']
#     train_y=train_df['gain_label']
#     test_pred_df=test_df.copy()#予測の結合先を作っておく
#     train_pred_df=train_df.copy()#予測の結合先を作っておく

#     #ざっと学習
#     rc = RandomForestClassifier(n_jobs=8, random_state=7,n_estimators=100,max_depth=10)
#     rc.fit(train_x,train_y)
#     pickle_path="../bot_database/{place_name}/model_pickle_{place_name}/model_selection_{place_name}_{V}.sav".format(place_name=place_name,V=version)#モデルを保存
#     #pickle_path="check_selection.sav"
#     pickle.dump(rc, open(pickle_path, "wb"))#モデルの保存
#     clf=pickle.load(open(pickle_path, 'rb'))#モデルを格納読み込む

#     #予測を出力(確率分布)
#     train_pred_proba_arr=clf.predict_proba(train_x)
#     train_pred_proba=[arr[1] for arr in train_pred_proba_arr]#確率分布の二次元配列なので[1]の部分だけ取り出す
#     test_pred_proba_arr = clf.predict_proba(test_x)
#     test_pred_proba=[arr[1] for arr in test_pred_proba_arr]#確率分布の二次元配列なので[1]の部分だけ取り出す
#     test_pred_df["pred_proba"]=test_pred_proba#予測を結合
#     train_pred_df["pred_proba"]=train_pred_proba#予測を結合

#     #確率分布の閾値で予測をバイナリに変換(閾値はいったん決めで0.8にする(引数で簡単に変えられるからよろしくうううう))
#     test_pred_df=pred_th_trans(test_pred_df,th)
#     train_pred_df=pred_th_trans(train_pred_df,th)
#     #モデルのスコアの計算
#     train_score=rc.score(train_x, train_y)
#     test_score=rc.score(test_x, test_y)
#     print("train:{}__test:{}".format(train_score,test_score))

#     col1='pred'
#     col2='gain_label'
#     cross_df,train_precision=making_cross(col1,col2 ,train_pred_df)
#     cross_df,test_precision=making_cross(col1,col2 ,test_pred_df)
#     print("CHECK::::train_precision:{}__test_precision:{}".format(train_precision,test_precision))#精度も一応算出しておく（閾値は0.8）

#     #使うモデルの選定を行う。
#     use_model_check_df=pd.DataFrame(columns=test_pred_df.columns)#最終的に使うと判定されたモデルのパラメータを格納するDF
#     selection_df=test_pred_df.copy()
#     selection_df['gain_year3']=gain_s#実際の収益の情報を追加する
#     selection_df=selection_df[selection_df['pred_proba']>=th].copy()#閾値はいったん決めで区切る
#     candidate_com=selection_df['target_com'].value_counts().index#閾値を超え、利益が出やすいと判断されたパラメータのあるcomを重複なく抜き出す
#     for com in candidate_com:
#         com_selection_df=selection_df[selection_df['target_com']==com].copy()
#         com_selection_df=com_selection_df.sort_values('pred_proba', ascending=False).iloc[:1]#各組の一番probaが高かったものを残す。
#         use_model_check_df=pd.concat([use_model_check_df, com_selection_df], axis=0)


#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================
#     #選定モデルを使っての本番で使うパラメータの選定パート============================================================================================================================================================================

#     #expand_score_df:::モデル選定のためのモデル作成の学習データ用の三年間を横に展開したスコアシート
#     #score_df:::実際にも本番で使うモデルを選ぶためのシート（答えなし）
#     has_proba_score_df=score_df.copy()
#     #予測を出力(確率分布)
#     pred_proba_arr = clf.predict_proba(score_df)
#     pred_proba=[arr[1] for arr in pred_proba_arr]#確率分布の二次元配列なので[1]の部分だけ取り出す
#     has_proba_score_df["pred_proba"]=pred_proba#予測を結合

#     #確率分布の閾値で予測をバイナリに変換(閾値はいったん決めで0.8にする(引数で簡単に変えられるからよろしくうううう))
#     has_proba_score_df=pred_th_trans(has_proba_score_df,th)

#     #使うモデルの選定を行う。
#     use_model_df=pd.DataFrame(columns=has_proba_score_df.columns)#最終的に使うと判定されたモデルのパラメータを格納するDF
#     selection_df=has_proba_score_df.copy()
#     selection_df=selection_df[selection_df['pred_proba']>=th].copy()#閾値はいったん決めで区切る
#     candidate_com=selection_df['target_com'].value_counts().index#閾値を超え、利益が出やすいと判断されたパラメータのあるcomを重複なく抜き出す
#     display(selection_df)
#     for com in candidate_com:
#         com_selection_df=selection_df[selection_df['target_com']==com].copy()
#         com_selection_df=com_selection_df.sort_values('pred_proba', ascending=False).iloc[:1]#各組の一番probaが高かったものを残す。
#         use_model_df=pd.concat([use_model_df, com_selection_df], axis=0)
#     #dir_path =  "../bot_database/{place_name}/model_score_{place_name}/use_model/use_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#選定されたモデルのリストを出力
#     use_model_df.to_csv(dir_path, encoding='utf_8_sig')


#     return use_model_check_df,use_model_df#決まったパラメータでモデルを作成できてるかのチェック

In [8]:
# #データ格納用のディレクトリ作り
# version='V3_1'#バージョン
# years=[2018,2019,2020]#使用する三年分のデータ（古い順で！！！）

# place_master=master.get_place_master()
# for place in place_master.items():
#     place_name=place[1]
#     print('=========================================================================================================================')
#     print(place_name)
#     ################################################========================================================================================================================
#     #各会場の学習データの読み込みと加工
#     expand_filepath="../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_3year_expand_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
#     expand_score_df=pd.read_csv(expand_filepath)#3年分のデータの読み込み
#     expand_score_df=expand_score_df.drop(["Unnamed: 0"],axis=1)#余計な列の削除
#     input_expand_score=len(expand_score_df)#データの件数
#     expand_score_df=expand_score_df.dropna()#収益の計算ができない部分がnanになって排出されてる臭いので削除
#     after_expand_score=len(expand_score_df)#データの件数
    
#     score_filepath="../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
#     score_df=pd.read_csv(score_filepath)#２年分の本番予測用のデータ
#     input_score=len(score_df)#データの件数
#     score_df=score_df.drop(["Unnamed: 0"],axis=1)#余計な列の削除
#     score_df=score_df.drop(["gain_year3"],axis=1)#余計な列の削除
#     score_df=score_df.dropna()#収益の計算ができない部分がnanになって排出されてる臭いので削除
#     after_score=len(score_df)#データの件数
#     print("input_expand:{}_____droped_expand:{}".format(input_expand_score,after_expand_score))
#     print("input:{}_____droped:{}".format(input_score,after_score))
#     #select.model_selection_save(expand_score_df,score_df,place_name,version,th=0.75)
#     model_selection_save(expand_score_df,score_df,place_name,version,th=0.65)